
- https://github.com/jayroxis/PINNs/blob/master/Burgers%20Equation/Burgers%20Inference%20(PyTorch).ipynb

In [2]:
import torch
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import warnings

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [3]:
def figsize(scale, nplots = 1):
    fig_width_pt = 390.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = nplots*fig_width*golden_mean       # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(1.0),     # default fig size of 0.9 textwidth
    "pgf.preamble": r"\usepackage[utf8x]{inputenc}, \usepackage[T1]{fontenc}",
    # [
    #     r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
    #     r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
    #     ]
    }

mpl.rcParams.update(pgf_with_latex)

import matplotlib.pyplot as plt

# I make my own newfig and savefig functions
def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename, crop = True):
    if crop == True:
#        plt.savefig('{}.pgf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.eps'.format(filename), bbox_inches='tight', pad_inches=0)
    else:
#        plt.savefig('{}.pgf'.format(filename))
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
class PhysicsInformedNN():
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x_u = torch.tensor(X_u[:, 0:1], requires_grad=True).float().to(device)
        self.t_u = torch.tensor(X_u[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.t_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        self.layers = layers
        self.nu = nu
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        # self.optimizer = torch.optim.LBFGS(
        #     self.dnn.parameters(), 
        #     lr=1.0, 
        #     max_iter=50000, 
        #     max_eval=50000, 
        #     history_size=50,
        #     tolerance_grad=1e-5, 
        #     tolerance_change=1.0 * np.finfo(float).eps,
        #     line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        # )
        self.optimizer = torch.optim.Adam(self.dnn.parameters(), lr=1e-3)
        self.iter = 0
        
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        u = self.net_u(x, t)
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = u_t + u * u_x - self.nu * u_xx
        return f
    
    # def loss_func(self):
    #     self.optimizer.zero_grad()
        
    #     u_pred = self.net_u(self.x_u, self.t_u)
    #     f_pred = self.net_f(self.x_f, self.t_f)
    #     loss_u = torch.mean((self.u - u_pred) ** 2)
    #     loss_f = torch.mean(f_pred ** 2)
        
    #     loss = loss_u + loss_f
        
    #     loss.backward()
    #     self.iter += 1
    #     if self.iter % 100 == 0:
    #         print(
    #             'Iter %d, Loss: %.2e, Loss_u: %.2e, Loss_f: %.2e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
    #         )
    #     return loss
    
    # def train(self):
    #     self.dnn.train()
                
    #     # Backward and optimize
    #     self.optimizer.step(self.loss_func)

    def train(self, n_epochs):
        for epoch in range(1, n_epochs + 1):
            u_pred = self.net_u(self.x_u, self.t_u)
            f_pred = self.net_f(self.x_f, self.t_f)
            loss_u = torch.mean((self.u - u_pred) ** 2)
            loss_f = torch.mean(f_pred ** 2)
            
            loss = loss_u + loss_f
            
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            if epoch % (n_epochs // 20) == 0:
                print(f"[{epoch:4d}/{n_epochs}] loss: {loss.item():.2e} "
                      f"loss_u: {loss_u:.2e} loss_f: {loss_f:.2e}")
            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [51]:
from pyDOE import lhs

nu = 0.01/np.pi
noise = 0.0        

N_u = 100
N_f = 10000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat('burgers_shock.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)    

xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

In [52]:
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)
model.train(2000)

[ 100/2000] loss: 1.43e-01 loss_u: 1.32e-01 loss_f: 1.12e-02
[ 200/2000] loss: 8.71e-02 loss_u: 6.34e-02 loss_f: 2.37e-02
[ 300/2000] loss: 6.04e-02 loss_u: 4.39e-02 loss_f: 1.65e-02
[ 400/2000] loss: 4.23e-02 loss_u: 3.04e-02 loss_f: 1.19e-02
[ 500/2000] loss: 2.73e-02 loss_u: 1.86e-02 loss_f: 8.69e-03
[ 600/2000] loss: 2.09e-02 loss_u: 1.30e-02 loss_f: 7.95e-03
[ 700/2000] loss: 1.82e-02 loss_u: 1.19e-02 loss_f: 6.22e-03
[ 800/2000] loss: 1.58e-02 loss_u: 9.91e-03 loss_f: 5.91e-03
[ 900/2000] loss: 1.47e-02 loss_u: 8.78e-03 loss_f: 5.94e-03
[1000/2000] loss: 1.23e-02 loss_u: 7.56e-03 loss_f: 4.69e-03
[1100/2000] loss: 1.05e-02 loss_u: 6.15e-03 loss_f: 4.37e-03
[1200/2000] loss: 9.37e-03 loss_u: 5.23e-03 loss_f: 4.14e-03
[1300/2000] loss: 1.16e-02 loss_u: 4.95e-03 loss_f: 6.61e-03
[1400/2000] loss: 7.90e-03 loss_u: 4.26e-03 loss_f: 3.64e-03
[1500/2000] loss: 7.83e-03 loss_u: 4.23e-03 loss_f: 3.59e-03
[1600/2000] loss: 6.96e-03 loss_u: 3.68e-03 loss_f: 3.28e-03
[1700/2000] loss: 8.85e-

In [53]:
u_pred, f_pred = model.predict(X_star)

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
print('Error u: %e' % (error_u))                     

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact - U_pred)

Error u: 2.202386e-01


In [60]:
plt.contourf(X, T, U_pred)

Error in callback <function _draw_all_if_interactive at 0x7f5412c20f70> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 539.643x333.518 with 1 Axes>

In [54]:
""" The aesthetic setting has changed. """

####### Row 0: u(t,x) ##################    

fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot(111)

h = ax.imshow(U_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[t.min(), t.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.10)
# cbar = fig.colorbar(h, cax=cax)
# cbar.ax.tick_params(labelsize=15) 

# ax.plot(
#     X_u_train[:,1], 
#     X_u_train[:,0], 
#     'kx', label = 'Data (%d points)' % (u_train.shape[0]), 
#     markersize = 4,  # marker size doubled
#     clip_on = False,
#     alpha=1.0
# )

# line = np.linspace(x.min(), x.max(), 2)[:,None]
# ax.plot(t[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
# ax.plot(t[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
# ax.plot(t[75]*np.ones((2,1)), line, 'w-', linewidth = 1)

# ax.set_xlabel('$t$', size=20)
# ax.set_ylabel('$x$', size=20)
# ax.legend(
#     loc='upper center', 
#     bbox_to_anchor=(0.9, -0.05), 
#     ncol=5, 
#     frameon=False, 
#     prop={'size': 15}
# )
# ax.set_title('$u(t,x)$', fontsize = 20) # font size doubled
# ax.tick_params(labelsize=15)

plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 900x500 with 1 Axes>